# Custom Layers

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [5]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

#### Classic model

In [34]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4997 - accuracy: 0.8250 - val_loss: 0.4413 - val_accuracy: 0.8431
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3761 - accuracy: 0.8633 - val_loss: 0.3931 - val_accuracy: 0.8612
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3366 - accuracy: 0.8769 - val_loss: 0.3702 - val_accuracy: 0.8674
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3109 - accuracy: 0.8867 - val_loss: 0.3631 - val_accuracy: 0.8684
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.3568 - accuracy: 0.8708


[0.35676950216293335, 0.8708000183105469]

## Lambda layer

#### Custom Activation Function

With custom layer we still don't have the controll to set up the `activation` function, therefore we use `Lambda` function to workaround.

We create our `own relu finction`.

`tf.keras.layers.Dense(128),
tf.keras.layers.Lambda(lambda x: tf.abs(x))`

In [8]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5148 - accuracy: 0.8227 - val_loss: 0.4602 - val_accuracy: 0.8368
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3932 - accuracy: 0.8606 - val_loss: 0.4253 - val_accuracy: 0.8534
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3572 - accuracy: 0.8725 - val_loss: 0.4141 - val_accuracy: 0.8607
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3330 - accuracy: 0.8818 - val_loss: 0.3867 - val_accuracy: 0.8647
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.4004 - accuracy: 0.8612


[0.4003850221633911, 0.8611999750137329]

---

#### Custom Activation Function 

Calling custom function

`tf.keras.layers.Dense(128),
tf.keras.layers.Lambda(my_relu),`

In [14]:
def my_relu(x):
    return tf.keras.backend.maximum(-0.1, x)

In [15]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4895 - accuracy: 0.8270 - val_loss: 0.4318 - val_accuracy: 0.8488
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3716 - accuracy: 0.8659 - val_loss: 0.3923 - val_accuracy: 0.8557
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3327 - accuracy: 0.8781 - val_loss: 0.3793 - val_accuracy: 0.8620
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3105 - accuracy: 0.8855 - val_loss: 0.3491 - val_accuracy: 0.8746
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3595 - accuracy: 0.8688


[0.3594619929790497, 0.8687999844551086]

---

## Custom layer

With custom layer we still don't have the controll to set up the `activation` function, therefore we use `Lambda` function to workaround.

In [2]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self,units=32):
        super(CustomLayer,self).__init__()
        self.units=units
    def build(self,input_shape):
        w_init=tf.random_normal_initializer()
        self.w=tf.Variable(name='kernel',initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
        b_init=tf.zeros_initializer()
        self.b=tf.Variable(name='bias',initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)
    def call(self,inputs):
        return tf.matmul(inputs,self.w)+ self.b

In [3]:
def my_relu(x):
    return tf.keras.backend.maximum(-0.1, x)

In [6]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    CustomLayer(128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.4891 - accuracy: 0.8272 - val_loss: 0.4144 - val_accuracy: 0.8542
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3738 - accuracy: 0.8636 - val_loss: 0.4071 - val_accuracy: 0.8544
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3359 - accuracy: 0.8775 - val_loss: 0.3613 - val_accuracy: 0.8682
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3110 - accuracy: 0.8852 - val_loss: 0.3692 - val_accuracy: 0.8635
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.3440 - accuracy: 0.8796


[0.344041645526886, 0.8795999884605408]

In [9]:
layer=CustomLayer(1)

x=tf.ones((1,1))
y=layer(x)

layer.variables

[<tf.Variable 'custom_layer_4/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.06907438]], dtype=float32)>,
 <tf.Variable 'custom_layer_4/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

---

In [28]:
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0])
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0])

layer=CustomLayer(1)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1,input_shape=[1])
])

model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500)

print(model.predict([10.0]))
print(model.variables)

Epoch 1/500
1/1 [==============================] - 1s 507ms/step - loss: 35.7622
Epoch 2/500
1/1 [==============================] - 0s 10ms/step - loss: 28.4735
Epoch 3/500
1/1 [==============================] - 0s 8ms/step - loss: 22.7321
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - loss: 18.2084
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - loss: 14.6426
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 11.8307
Epoch 7/500
1/1 [==============================] - 0s 7ms/step - loss: 9.6121
Epoch 8/500
1/1 [==============================] - 0s 6ms/step - loss: 7.8604
Epoch 9/500
1/1 [==============================] - 0s 7ms/step - loss: 6.4760
Epoch 10/500
1/1 [==============================] - 0s 3ms/step - loss: 5.3809
Epoch 11/500
1/1 [==============================] - 0s 0s/step - loss: 4.5135
Epoch 12/500
1/1 [==============================] - 0s 7ms/step - loss: 3.8252
Epoch 13/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 10ms/step - loss: 0.2065
Epoch 105/500
1/1 [==============================] - 0s 6ms/step - loss: 0.2023
Epoch 106/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1981
Epoch 107/500
1/1 [==============================] - 0s 9ms/step - loss: 0.1941
Epoch 108/500
1/1 [==============================] - 0s 9ms/step - loss: 0.1901
Epoch 109/500
1/1 [==============================] - 0s 7ms/step - loss: 0.1862
Epoch 110/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1824
Epoch 111/500
1/1 [==============================] - 0s 8ms/step - loss: 0.1786
Epoch 112/500
1/1 [==============================] - 0s 0s/step - loss: 0.1749
Epoch 113/500
1/1 [==============================] - 0s 8ms/step - loss: 0.1714
Epoch 114/500
1/1 [==============================] - 0s 5ms/step - loss: 0.1678
Epoch 115/500
1/1 [==============================] - 0s 7ms/step - loss: 0.1644
Epoch 116/500
1/1 [==============================] - 0

1/1 [==============================] - 0s 7ms/step - loss: 0.0249
Epoch 207/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0244
Epoch 208/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0239
Epoch 209/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0234
Epoch 210/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0229
Epoch 211/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0224
Epoch 212/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0220
Epoch 213/500
1/1 [==============================] - 0s 4ms/step - loss: 0.0215
Epoch 214/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0211
Epoch 215/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0206
Epoch 216/500
1/1 [==============================] - 0s 0s/step - loss: 0.0202
Epoch 217/500
1/1 [==============================] - 0s 1ms/step - loss: 0.0198
Epoch 218/500
1/1 [==============================] -

1/1 [==============================] - 0s 12ms/step - loss: 0.0030
Epoch 309/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0029
Epoch 310/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0029
Epoch 311/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0028
Epoch 312/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0028
Epoch 313/500
1/1 [==============================] - 0s 6ms/step - loss: 0.0027
Epoch 314/500
1/1 [==============================] - 0s 5ms/step - loss: 0.0026
Epoch 315/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0026
Epoch 316/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 317/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0025
Epoch 318/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0024
Epoch 319/500
1/1 [==============================] - 0s 0s/step - loss: 0.0024
Epoch 320/500
1/1 [==============================] 

1/1 [==============================] - 0s 3ms/step - loss: 3.7574e-04
Epoch 409/500
1/1 [==============================] - 0s 6ms/step - loss: 3.6802e-04
Epoch 410/500
1/1 [==============================] - 0s 5ms/step - loss: 3.6047e-04
Epoch 411/500
1/1 [==============================] - 0s 4ms/step - loss: 3.5306e-04
Epoch 412/500
1/1 [==============================] - 0s 3ms/step - loss: 3.4581e-04
Epoch 413/500
1/1 [==============================] - 0s 6ms/step - loss: 3.3870e-04
Epoch 414/500
1/1 [==============================] - 0s 3ms/step - loss: 3.3175e-04
Epoch 415/500
1/1 [==============================] - 0s 10ms/step - loss: 3.2493e-04
Epoch 416/500
1/1 [==============================] - 0s 8ms/step - loss: 3.1826e-04
Epoch 417/500
1/1 [==============================] - 0s 4ms/step - loss: 3.1172e-04
Epoch 418/500
1/1 [==============================] - 0s 15ms/step - loss: 3.0532e-04
Epoch 419/500
1/1 [==============================] - 0s 2ms/step - loss: 2.9905e-04
Epoc

In [27]:
# xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0])
# ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0])

# layer=CustomLayer(1)
# model = tf.keras.models.Sequential([
#     layer
# ])

# model.compile(optimizer='sgd', loss='mean_squared_error')
# model.fit(xs, ys, epochs=500,verbose=0)

# print(model.predict([10.0]))
# print(layer.variables)

## Custom Layer with activation

In [4]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self,units=32,activation='linear'):
        super(CustomLayer,self).__init__()
        self.units=units
        self.activation=tf.keras.activations.get(activation)
    def build(self,input_shape):
        w_init=tf.random_normal_initializer()
        self.w=tf.Variable(name='kernel',initial_value=w_init(shape=(input_shape[-1],self.units),dtype='float32'),trainable=True)
        b_init=tf.zeros_initializer()
        self.b=tf.Variable(name='bias',initial_value=b_init(shape=(self.units,),dtype='float32'),trainable=True)
    def call(self,inputs):
        # return activation function of a matmul
        return self.activation(tf.matmul(inputs,self.w)+ self.b)

In [48]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,)),
    CustomLayer(128,activation='softmax'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
model.fit(training_images,training_labels,epochs=5,validation_data=(test_images,test_labels))
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.4812 - accuracy: 0.5637 - val_loss: 0.9672 - val_accuracy: 0.7287
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.0156 - accuracy: 0.6215 - val_loss: 0.7266 - val_accuracy: 0.7452
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9364 - accuracy: 0.6334 - val_loss: 0.6523 - val_accuracy: 0.7480
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.9068 - accuracy: 0.6339 - val_loss: 0.6162 - val_accuracy: 0.7573
Epoch 5/5
313/313 [==============================] - 0s 1ms/step - loss: 0.5993 - accuracy: 0.7728


[0.5993068814277649, 0.7728000283241272]